In [1]:
# step by step guide: https://towardsdatascience.com/multiclass-classification-algorithm-from-scratch-with-a-project-in-python-step-by-step-guide-485a83c79992

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.experimental import enable_halving_search_cv
#from sklearn.model_selection import HalvingRandomSearchCV


In [3]:
#test_run = True
test_run = False

In [4]:
df = pd.read_csv("Temp/df_proc_train_set.csv")

In [5]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
if test_run:
    df, _, __, ___ = train_test_split(df, df['sol_category'], train_size=500, stratify=df['sol_category'])

In [7]:
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
#y = pd.read_excel(xl, 'y', header = None)

In [9]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [10]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

70711


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Only exfp

In [11]:
'''X_ecfp = df.copy(deep=True)
drop_list_desc = [column for column in X_ecfp.columns if (column.startswith('example_') or column.startswith('rdkit_desc_'))]

# Drop features 
X_ecfp.drop(drop_list_desc, axis=1, inplace=True)
X_ecfp.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_ecfp'''

"X_ecfp = df.copy(deep=True)\ndrop_list_desc = [column for column in X_ecfp.columns if (column.startswith('example_') or column.startswith('rdkit_desc_'))]\n\n# Drop features \nX_ecfp.drop(drop_list_desc, axis=1, inplace=True)\nX_ecfp.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)\nX_ecfp"

#### Only descriptors

In [12]:
'''X_desc = df.copy(deep=True)
drop_list_MFP = [column for column in X_desc.columns if column.startswith('ecfp_')]

# Drop features 
X_desc.drop(drop_list_MFP, axis=1, inplace=True)
X_desc.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_desc'''

"X_desc = df.copy(deep=True)\ndrop_list_MFP = [column for column in X_desc.columns if column.startswith('ecfp_')]\n\n# Drop features \nX_desc.drop(drop_list_MFP, axis=1, inplace=True)\nX_desc.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)\nX_desc"

#### only example descriptors

In [13]:
'''X_example = X_desc.copy(deep=True)
drop_list_ex = [column for column in X_example.columns if column.startswith('rdkit_desc_')]

# Drop features 
X_example.drop(drop_list_ex, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_example'''

"X_example = X_desc.copy(deep=True)\ndrop_list_ex = [column for column in X_example.columns if column.startswith('rdkit_desc_')]\n\n# Drop features \nX_example.drop(drop_list_ex, axis=1, inplace=True)\n#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)\nX_example"

#### only rdkit descriptors

In [14]:
'''X_rdd = X_desc.copy(deep=True)
drop_list_rd = [column for column in X_rdd.columns if column.startswith('example_')]

# Drop features 
X_rdd.drop(drop_list_rd, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_rdd'''

"X_rdd = X_desc.copy(deep=True)\ndrop_list_rd = [column for column in X_rdd.columns if column.startswith('example_')]\n\n# Drop features \nX_rdd.drop(drop_list_rd, axis=1, inplace=True)\n#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)\nX_rdd"

In [15]:
y = df.iloc[:, 2]

In [16]:
#y

In [17]:
y.unique()

array([0, 1, 2], dtype=int8)

## multi-class logistic regression

In [18]:
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from numpy import linspace

In [19]:
model = LogisticRegression(random_state=0, max_iter=2000)#, C=0.9448979591836735)

#'''
parameters = {
    #'max_iter' : [2000], #range(1000, 2000),
    #'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    #'penalty' : ['l2', 'none'],
    'C' : linspace(0.1, 1.0, num=50)
    }
    #'''

In [20]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [21]:
all_dataset = list()
all_dataset.append(X)
#all_dataset.append(X_ecfp)
#all_dataset.append(X_desc)
#all_dataset.append(X_example)
#all_dataset.append(X_rdd)

qwck_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

#print('------------------------------------------------------')

for dataset in all_dataset:
    # apply StandardScaler
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)
    dataset = scaler.fit_transform(dataset)

    # split dataset
    StratifiedKFold(n_splits=5)
    X_train, X_val, y_train, y_val = train_test_split(dataset, y, random_state=1,  shuffle=True, stratify=y)#, test_size=0.2, train_size=0.8)

    # define class weights
    from sklearn.utils import class_weight
    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced',
        y=y_train
        )

    # method
    #'''
    randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters, cv = 5, n_iter = 10, n_jobs=5, scoring=qwck_scorer, verbose=2)
    randm_src.fit(X_train, y_train, sample_weight=classes_weights)
    #'''
    #model.fit(X_train, y_train, sample_weight=classes_weights)

    

    #'''
    # output
    print(f'results of data set {dataset}')
    print('------------------------------------------------------')
    print(" Results from Random Search: " )
    print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
    print("\n The best score across ALL searched params:\n", randm_src.best_score_)
    print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')
    #'''


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ..............................C=0.21020408163265308; total time= 7.1min
[CV] END ..............................C=0.21020408163265308; total time= 7.3min
[CV] END ..............................C=0.21020408163265308; total time= 7.5min
[CV] END ..............................C=0.21020408163265308; total time= 7.6min
[CV] END ..............................C=0.21020408163265308; total time= 7.8min
[CV] END ...............................C=0.8714285714285714; total time=13.8min
[CV] END ...............................C=0.8714285714285714; total time=13.9min
[CV] END ...............................C=0.8714285714285714; total time=13.9min
[CV] END ...............................C=0.8714285714285714; total time=14.4min
[CV] END ...............................C=0.8714285714285714; total time=13.9min
[CV] END ...............................C=0.2285714285714286; total time= 8.1min
[CV] END ...............................C=0.2285

In [23]:
model = LogisticRegression(random_state=0, max_iter=2000, C=0.889795918367347)#, C=0.9448979591836735)
model.fit(X_train, y_train, sample_weight=classes_weights)

LogisticRegression(C=0.889795918367347, max_iter=2000, random_state=0)

In [24]:
valPredictions = model.predict(X_val)
    
#quad kappa score
sk_quad_kappa = cohen_kappa_score(y_val, valPredictions, weights='quadratic')

print(f'quad kappa score: {sk_quad_kappa}')

quad kappa score: 0.007166096289818125


In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, valPredictions, labels=[2, 1, 0])

array([[10933,  3251,  2275],
       [  449,   161,    99],
       [  329,   104,    77]])

In [ ]:
real_test_set = pd.read_csv('Temp/df_proc_test_set.csv')
sub_template = pd.read_csv('Data/submission_template_rdm.csv')
(sub_template['Id'] == real_test_set['Id']).value_counts()

True    30307
Name: Id, dtype: int64

In [ ]:
real_test_set.drop(columns=['Id', 'smiles'], inplace=True)

# apply StandardScaler
#scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(real_test_set)
#real_test_set = scaler.fit_transform(real_test_set)

# prediction
realtestPredictions = model.predict(real_test_set)
sub_template['pred'] = realtestPredictions

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
set(realtestPredictions)

In [ ]:
sub_template['pred'].value_counts()